In [2]:
# Built-in imports:
from typing import List
from itertools import islice

# Dependency imports: 
import tensorflow as tf
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

# Import the GravyFlow module:
import gravyflow as gf

In [3]:
# Setup the IFODataObtainer object:
# This object will be used to obtain real interferometer data based on specified parameters.
ifo_data_obtainer: gf.IFODataObtainer = gf.IFODataObtainer(
    observing_runs=gf.ObservingRun.O3, # Specify the observing run (e.g., O3).
    data_quality=gf.DataQuality.BEST,  # Choose the quality of the data (e.g., BEST).
    data_labels=[                      # Define the types of data to include.
        gf.DataLabel.NOISE, 
        gf.DataLabel.GLITCHES
    ],
    segment_order=gf.SegmentOrder.RANDOM, # Order of segment retrieval (e.g., RANDOM).
    force_acquisition=True,               # Force the acquisition of new data.
    cache_segments=False                  # Choose not to cache the segments.
)

In [4]:
# Initialize the noise generator wrapper:
# This wrapper will use the ifo_data_obtainer to generate real noise based on the specified parameters.
noise: gf.NoiseObtainer = gf.NoiseObtainer(
    ifo_data_obtainer=ifo_data_obtainer, # Use the previously set up IFODataObtainer object.
    noise_type=gf.NoiseType.REAL,        # Specify the type of noise as REAL.
    ifos=gf.IFO.L1                       # Specify the interferometer (e.g., LIGO Livingston L1).
)

In [5]:
def generate_noise():
    with gf.env():
        return next(noise(num_examples_per_batch=128))

In [6]:
# Use the TensorFlow environment 'env' created earlier with gf.env()
with gf.env():
	# Generate noise by calling the noise object with one example per batch.
	# The next() function retrieves the generated noise data.
	# This returns onsource noise, offsource noise, and GPS time of the noise segment.
	onsource, offsource, gps_times = next(noise(num_examples_per_batch=32))

In [7]:
# Generate a plot for the onsource noise strain.
# The first element of onsource (onsource[0]) is used for plotting,
# and the corresponding GPS time is included in the title.
onsource_gps_time = gps_times[0][0].numpy()
onsource_strain_plot = gf.generate_strain_plot(
    {"Onsource Noise": onsource[0]},
    title=f"Onsource Real Noise at {onsource_gps_time:.0f} s"
)

# Generate a plot for the offsource noise strain.
# The first element of offsource (offsource[0]) is used for plotting,
# and the corresponding GPS time is included in the title.
offsource_gps_time = gps_times[0][0].numpy() - 16.5
offsource_strain_plot = gf.generate_strain_plot(
    {"Offsource Noise": offsource[0]},
    title=f"Offsource Real Noise at {offsource_gps_time:.0f} s"
)

# Create a layout for the plots.
# This layout arranges the onsource and offsource strain plots side by side.
layout: List = [[onsource_strain_plot, offsource_strain_plot]]

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(layout)
output_notebook()
show(grid)

In [ ]:

with gf.env():
    while noise(num_examples_per_batch=32):
        pass